# Semantic Segmentation


In [1]:
import os.path
import tensorflow as tf
import helper
import warnings
from distutils.version import LooseVersion
import project_tests as tests

/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ec2-user/.config/matplotlib/matplotlibrc", line #2
  (fname, cnt))
/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ec2-user/.config/matplotlib/matplotlibrc", line #3
  (fname, cnt))


## Check for a GPU

In [2]:
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

Default GPU Device: /device:GPU:0


## Check TensorFlow Version


In [3]:
assert LooseVersion(tf.__version__) >= LooseVersion('1.0'), 'Please use TensorFlow version 1.0 or newer.  You are using {}'.format(tf.__version__)
print('TensorFlow Version: {}'.format(tf.__version__))

TensorFlow Version: 1.7.0


In [4]:
def load_vgg(sess, vgg_path):
    """
    Load Pretrained VGG Model into TensorFlow.
    :param sess: TensorFlow Session
    :param vgg_path: Path to vgg folder, containing "variables/" and "saved_model.pb"
    :return: Tuple of Tensors from VGG model (image_input, keep_prob, layer3_out, layer4_out, layer7_out)
    """
    # TODO: Implement function
    #   Use tf.saved_model.loader.load to load the model and weights
    vgg_tag = 'vgg16'
    vgg_input_tensor_name = 'image_input:0'
    vgg_keep_prob_tensor_name = 'keep_prob:0'
    vgg_layer3_out_tensor_name = 'layer3_out:0'
    vgg_layer4_out_tensor_name = 'layer4_out:0'
    vgg_layer7_out_tensor_name = 'layer7_out:0'

    # Refer https://stackoverflow.com/questions/45705070/how-to-load-and-use-a-saved-model-on-tensorflow
    tf.saved_model.loader.load(sess, [vgg_tag], vgg_path)
    graph = tf.get_default_graph()
    
    image_input = graph.get_tensor_by_name(vgg_input_tensor_name)
    keep_prob   = graph.get_tensor_by_name(vgg_keep_prob_tensor_name)
    layer3_out  = graph.get_tensor_by_name(vgg_layer3_out_tensor_name)
    layer4_out  = graph.get_tensor_by_name(vgg_layer4_out_tensor_name)
    layer7_out  = graph.get_tensor_by_name(vgg_layer7_out_tensor_name)
    
    return image_input, keep_prob, layer3_out, layer4_out, layer7_out

In [5]:
tests.test_load_vgg(load_vgg, tf)

Tests Passed


In [6]:
def layers(vgg_layer3_out, vgg_layer4_out, vgg_layer7_out, num_classes):
    """
    Create the layers for a fully convolutional network.  Build skip-layers using the vgg layers.
    :param vgg_layer3_out: TF Tensor for VGG Layer 3 output
    :param vgg_layer4_out: TF Tensor for VGG Layer 4 output
    :param vgg_layer7_out: TF Tensor for VGG Layer 7 output
    :param num_classes: Number of classes to classify
    :return: The Tensor for the last layer of output
    """

    std_dev = 0.001
    reg = 0.0001
    
    # 1x1 Convolutions

    conx_1x1_layer3 = tf.layers.conv2d(vgg_layer3_out, num_classes, 1,
                                       kernel_initializer = tf.random_normal_initializer(stddev = std_dev),
                                       kernel_regularizer = tf.contrib.layers.l2_regularizer(reg),
                                       name = "conx_1x1_layer3")

    conx_1x1_layer4 = tf.layers.conv2d(vgg_layer4_out, num_classes, 1,
                                       kernel_initializer = tf.random_normal_initializer(stddev = std_dev),
                                       kernel_regularizer = tf.contrib.layers.l2_regularizer(reg),
                                       name = "conx_1x1_layer4")

    conx_1x1_layer7 = tf.layers.conv2d(vgg_layer7_out, num_classes, 1,
                                       kernel_initializer = tf.random_normal_initializer(stddev = std_dev),
                                       kernel_regularizer = tf.contrib.layers.l2_regularizer(reg),
                                       name = "conx_1x1_layer7")      

    upsample_2x_l7 = tf.layers.conv2d_transpose(vgg_layer7_out, num_classes, 4, strides = (2, 2),
                                       kernel_initializer = tf.random_normal_initializer(stddev = std_dev),
                                       kernel_regularizer = tf.contrib.layers.l2_regularizer(reg),
                                       name = "upsample_2x_l7")
    
    fuse1 = tf.add(upsample_2x_l7, conx_1x1_layer4)

    upsample_2x_f1 = tf.layers.conv2d_transpose(fuse1, num_classes, 4, strides = (2, 2),
                                       kernel_initializer = tf.random_normal_initializer(stddev = std_dev),
                                       kernel_regularizer = tf.contrib.layers.l2_regularizer(reg),
                                       name = "upsample_2x_f1")
    
    fuse2 = tf.add(upsample_2x_f1, conx_1x1_layer3)
    
    upsample_2x_f2 = tf.layers.conv2d_transpose(fuse2, num_classes, 16, strides = (8, 8),
                                       kernel_initializer = tf.random_normal_initializer(stddev = std_dev),
                                       kernel_regularizer = tf.contrib.layers.l2_regularizer(reg),
                                       name = "upsample_2x_f2")
    
    return upsample_2x_f2

In [ ]:
tests.test_layers(layers)

In [9]:
def run():
    num_classes = 2
    image_shape = (160, 576)
    data_dir = './data'
    runs_dir = './runs'
    tests.test_for_kitti_dataset(data_dir)

    # Download pretrained vgg model
    helper.maybe_download_pretrained_vgg(data_dir)

    # OPTIONAL: Train and Inference on the cityscapes dataset instead of the Kitti dataset.
    # You'll need a GPU with at least 10 teraFLOPS to train on.
    #  https://www.cityscapes-dataset.com/

    with tf.Session() as sess:
        # Path to vgg model
        vgg_path = os.path.join(data_dir, 'vgg')
        # Create function to get batches
        get_batches_fn = helper.gen_batch_function(os.path.join(data_dir, 'data_road/training'), image_shape)

        # OPTIONAL: Augment Images for better results
        #  https://datascience.stackexchange.com/questions/5224/how-to-prepare-augment-images-for-neural-network

        # TODO: Build NN using load_vgg, layers, and optimize function
        input_image, keep_prob, layer3_out, layer4_out, layer7_out = load_vgg(sess, vgg_path)
        outputs = layers(layer3_out, layer4_out, layer7_out, num_classes)
        

        # TODO: Train NN using the train_nn function

        # TODO: Save inference data using helper.save_inference_samples
        helper.save_inference_samples(runs_dir, data_dir, sess, image_shape, logits, keep_prob, input_image)

        # OPTIONAL: Apply the trained model to a video

In [10]:
run()

0.00B [00:00, ?B/s]

Tests Passed


997MB [00:16, 58.8MB/s]                               


Extracting model...
INFO:tensorflow:Restoring parameters from b'./data/vgg/variables/variables'


NameError: name 'logits' is not defined